Analiza języka:
W tym dokumencie przeprowadzam analizę języka angielskiego, polskiego i niemieckiego do języka na stronach internetowych (głównie https://bulbapedia.bulbagarden.net/wiki/). Wzór to jest suma wszystkich: dicti.get(word, 0)*(1 - abs(ilosc_slow[word] - dicti.get(word, 0))), gdzie dicti to zbiór k>0 słów języka en/pl/de, a ilosc_slow to zbiór wszystkich słów na danej stronie (w głównej części), obie znormalizowane przez wspólną największą wartościowo w dicti częstotliwość słowa (odpowiednio dla każdego słownika), jeśli nie posiadają żadnego wspólnego słowa to prawdopodobieńtwo jest wyzerowane. W teście sprawdzam różne wartości k={3, 10, 100, 1000}, na stronach z wieloma słowami z wiki: https://bulbapedia.bulbagarden.net/wiki/Abomasnow_(Pok%C3%A9mon), spośród 50 randomowych stron z wiki stronę z najgorszym wynikiem funkcji: "https://bulbapedia.bulbagarden.net/wiki/Category:Pok%C3%A9mon_by_body_size", artykułów stron rządkowych (gov) odpowiednio Wielkiej Brytanii, Polski i Niemiec. 

In [ ]:
from wordfreq import zipf_frequency, top_n_list
import matplotlib.pyplot as plt
from collections import defaultdict
import requests
import json
import time
import re
from bs4 import BeautifulSoup
class Count_words:
    @staticmethod
    def count_words(strona, co1, co2):
        dictionary = Count.count(strona, co1, co2)
        with open("word-counts" + '.json', 'w', encoding="utf-8") as plik:
            json.dump(dictionary, plik, ensure_ascii=False, indent=2)

class Count:
    @staticmethod
    def count(strona, co1, co2):
        response = requests.get(strona)
        dicti = {}
        if response.status_code == 200: 
            soup = BeautifulSoup(response.text, 'html.parser')
            text_paragraphs = soup.find(co2, class_=co1).text
            ile = 0
            words = re.findall(r"\b\w+\b", text_paragraphs)
            for i, word in enumerate(words):
                dicti[word] = dicti.get(word, 0) + 1
            return dicti
        else:
            print("Strona nie istnieje")
            return dicti

def language_words_with_frequency(jezyk, ile):
    words = top_n_list(jezyk, ile)
    dicti = {}
    for i, word in enumerate(words):
        dicti[word] = zipf_frequency(word, jezyk)
    return dicti

def znormalizuj(ilosc_slow, dicti):
    ile = 0
    ile2 = 0
    for word in ilosc_slow:
        if dicti.get(word, 0) != 0:
            if ile <  dicti.get(word, 0):
                ile = dicti.get(word, 0)
                ile2 = ilosc_slow[word]
    if ile != 0:
        for word in ilosc_slow:
            ilosc_slow[word] = ilosc_slow[word]/ile2
            if dicti.get(word, 0) != 0:  dicti[word] = dicti.get(word, 0)/ile
    else:
        for word in ilosc_slow:
            if dicti.get(word, 0) != 0: dicti[word] = 0
    return ilosc_slow, dicti

def lang_confidence_score(ilosc_slow, dicti):
    ilosc_slow, dicti = znormalizuj(ilosc_slow, dicti)
    prawdopodobienstwo = 0
    slowo = 0
    for word in ilosc_slow:
        if ilosc_slow[word] > slowo: slowo = ilosc_slow[word]
    for word in ilosc_slow:
        ilosc_slow[word] = ilosc_slow[word]/slowo
    for word in ilosc_slow:
        prawdopodobienstwo += dicti.get(word, 0)*(1 - abs(ilosc_slow[word] - dicti.get(word, 0)))
    return prawdopodobienstwo




def testy():
    results = []
    Count_words.count_words("https://bulbapedia.bulbagarden.net/wiki/Abomasnow_(Pok%C3%A9mon)", 'mw-body-content', 'div')
    with open("word-counts" + '.json', 'r', encoding="utf-8") as plik:
        dicti_wiki_dlugi = json.load(plik)
    Count_words.count_words("https://bulbapedia.bulbagarden.net/wiki/Category:Pok%C3%A9mon_by_body_size", 'mw-body-content', 'div')
    with open("word-counts" + '.json', 'r', encoding="utf-8") as plik:
        dicti_wiki_zly_wynik = json.load(plik)
    Count_words.count_words("https://www.gov.uk/government/news/greater-access-to-breakthrough-trials-for-rare-cancer-patients", 'govuk-main-wrapper', 'main')
    with open("word-counts" + '.json', 'r', encoding="utf-8") as plik:
        dicti_nie_wiki_en = json.load(plik)
    Count_words.count_words("https://www.bundesregierung.de/breg-de/service/newsletter-und-abos/bulletin/bk-regierungserklaerung-2405056", 'bpa-container', 'div')
    with open("word-counts" + '.json', 'r', encoding="utf-8") as plik:
        dicti_nie_wiki_de = json.load(plik)
    Count_words.count_words("https://www.gov.pl/web/baza-wiedzy/prawo-o-dostepnosci-cyfrowej", "article-area main-container", 'div')
    with open("word-counts" + '.json', 'r', encoding="utf-8") as plik:
        dicti_nie_wiki_pl = json.load(plik)
    k_lista = [3, 10, 100, 1000]
    for k in k_lista:
        #dicti_wiki_dlugi
        score = lang_confidence_score(dicti_wiki_dlugi, language_words_with_frequency("en", k))
        results.append({
            "text": "dicti_wiki_dlugi",
            "language": "en",
            "k": k,
            "score": score
        })
        score = lang_confidence_score(dicti_wiki_dlugi, language_words_with_frequency("pl", k))
        results.append({
            "text": "dicti_wiki_dlugi",
            "language": "pl",
            "k": k,
            "score": score
        })
        score = lang_confidence_score(dicti_wiki_dlugi, language_words_with_frequency("de", k))
        results.append({
            "text": "dicti_wiki_dlugi",
            "language": "de",
            "k": k,
            "score": score
        })
        # dicti_wiki_zly_wynik
        score = lang_confidence_score(dicti_wiki_zly_wynik, language_words_with_frequency("en", k))
        results.append({
            "text": "dicti_wiki_zly_wynik",
            "language": "en",
            "k": k,
            "score": score
        })
        score = lang_confidence_score(dicti_wiki_zly_wynik, language_words_with_frequency("pl", k))
        results.append({
            "text": "dicti_wiki_zly_wynik",
            "language": "pl",
            "k": k,
            "score": score
        })
        score = lang_confidence_score(dicti_wiki_zly_wynik, language_words_with_frequency("de", k))
        results.append({
            "text": "dicti_wiki_zly_wynik",
            "language": "de",
            "k": k,
            "score": score
        })
        # dicti_nie_wiki_en
        score = lang_confidence_score(dicti_nie_wiki_en, language_words_with_frequency("en", k))
        results.append({
            "text": "dicti_nie_wiki_en",
            "language": "en",
            "k": k,
            "score": score
        })
        score = lang_confidence_score(dicti_nie_wiki_en, language_words_with_frequency("pl", k))
        results.append({
            "text": "dicti_nie_wiki_en",
            "language": "pl",
            "k": k,
            "score": score
        })
        score = lang_confidence_score(dicti_nie_wiki_en, language_words_with_frequency("de", k))
        results.append({
            "text": "dicti_nie_wiki_en",
            "language": "de",
            "k": k,
            "score": score
        })
        # dicti_nie_wiki_pl
        score = lang_confidence_score(dicti_nie_wiki_pl, language_words_with_frequency("en", k))
        results.append({
            "text": "dicti_nie_wiki_pl",
            "language": "en",
            "k": k,
            "score": score
        })
        score = lang_confidence_score(dicti_nie_wiki_pl, language_words_with_frequency("pl", k))
        results.append({
            "text": "dicti_nie_wiki_pl",
            "language": "pl",
            "k": k,
            "score": score
        })
        score = lang_confidence_score(dicti_nie_wiki_pl, language_words_with_frequency("de", k))
        results.append({
            "text": "dicti_nie_wiki_pl",
            "language": "de",
            "k": k,
            "score": score
        })
        # dicti_nie_wiki_de
        score = lang_confidence_score(dicti_nie_wiki_de, language_words_with_frequency("en", k))
        results.append({
            "text": "dicti_nie_wiki_de",
            "language": "en",
            "k": k,
            "score": score
        })
        score = lang_confidence_score(dicti_nie_wiki_de, language_words_with_frequency("pl", k))
        results.append({
            "text": "dicti_nie_wiki_de",
            "language": "pl",
            "k": k,
            "score": score
        })
        score = lang_confidence_score(dicti_nie_wiki_de, language_words_with_frequency("de", k))
        results.append({
            "text": "dicti_nie_wiki_de",
            "language": "de",
            "k": k,
            "score": score
        })
    
    grouped = defaultdict(list)

    for r in results:
        key = (r["text"], r["language"])
        grouped[key].append((r["k"], r["score"]))
    
    language_colors = {
        "en": "-",
        "pl": "--",
        "de": ":"
    }

    text_styles = {
        "dicti_wiki_dlugi": "tab:blue",
        "dicti_wiki_zly_wynik": "tab:orange",
        "dicti_nie_wiki_en": "tab:green",
        "dicti_nie_wiki_pl": "tab:purple",
        "dicti_nie_wiki_de": "tab:red"
    }
    plt.figure(figsize=(10, 6))
    
    for (text, lang), vals in grouped.items():
        vals.sort()
        ks, scores = zip(*vals)
        plt.plot(ks, scores, color=text_styles[text], linestyle=language_colors[lang], marker="o", label=f"{text} / {lang}")
    
    plt.xscale("log")
    plt.xlabel("k (top words)")
    plt.ylabel("confidence score")
    plt.title("Language confidence vs k")
    plt.legend()
    plt.grid(True)
    plt.show()

testy()

Wyniki:
Jak widać, im większe k tym większe prawdopodobieństwo przynależenia do języka, ze względu na najprawdopodobniej małą odmianę słów w angielskim, jak widać artykuły (oprócz tego specjalnie wybranego jako złośliwego) mają bardzo dobre wyniki przynależenia (zgodnie z prawdą). Artykuł złośliwy był bardzo krótki (więcej niż 20 słów), stąd małe przynależenie słownictwa i niski wynik przynależenia, artykuł nie był bardzo trudny do znalezienia, widać spore zmiany w wartościach prawdopodobieństwa i szukając wsród krótkich artykułów znalezienie złego wyniku nie było zbyt trudne. Generalnie widać dużą poprawę przynależenia do jezyka wraz z zwiększeniem k w angielskim i polskim, ale nie niemieckim, co zaskakujące największy skok był przy k=100, tymczasem dla k=1000 nie było dużej poprawy, zarazem słowniki języka polskiego i angielskiego zaczęły mieć wspólne słowa z artykułem po niemiecku, co w pewnien sposób pogarsza wynik. Polski język ma również spory wzrost wraz z k, jednak mniejszy niż angielski, zapewne przez sporą ilość odmian słów w polskim i rozróżnialność tego samego słowa w różnych odmianach jako dwa różne słowa. Po wynikach języków po których artykuł nie został napisany widać, że język angielski ma najwięcej wspólnych słów z niemieckim i polskim, następnie polski ma sporo podobieństw z niemieckim i angielskim, najmniejsze podobieństwo ma język niemiecki. Jak widać dobór języków ma spore znaczenie, ze względu na podobne słowa między językami, oryginalne litery w słownictwie, odmiany słów w języku. 